#  download data

In [ ]:
import cdsapi
c = cdsapi.Client()
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'grib',
        'area': [
            -9, 15, -13,
            17,
        ],
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
        ],
        'variable': 'total_precipitation',
    },
    'Data.grib')

In [ ]:
import pygrib
import numpy as np
import matplotlib.pyplot as plt
import os

file=os.path.abspath(os.getcwd())
file=file+'/Data.grib'
#file = '/home/az/Bureau/MFI_Test/Data.grib' # fix your path
if os.path.isfile(file) :
    
    gr = pygrib.open(file)
else :
    print("put the correcte path in file ")
gr.rewind() # rewind the iterator
Liste_Value=[] # put data in list
Liste_SumVal=[] # compte the sum of each (day)
for msg in gr:
    temps_vals=msg.values
    Liste_Value.append(temps_vals)
    Liste_SumVal.append(np.sum(temps_vals))

All_Value=np.array(Liste_Value)
All_SumVal=np.array(Liste_SumVal)

#  Onset for the  All area between 1981-2012

In [ ]:
# convert m3/m3 to mm
def id2precip(Input):
    temp=Input*100
    return 0.0014*temp*temp+0.7166*temp

def id2precip2(Input):
    return Input*1000

In [ ]:
## camput AA for all aera : 
A=id2precip2(All_Value)
B=id2precip2(All_SumVal)

import numpy as np 
def AA(Input):
    Moy=np.mean(Input)
    aa=[]
    for i in range(1,366):
        aa.append(sum(Input[0:i]-Moy))
    return np.array(aa)


def plotAA(Input,year):
    plt.plot(Input, label=year)    
    #end=np.argmax(Input)
    #start=np.argmin(Input)
    #plt.plot([start,start],[Input[start],Input[end]],color='black')  
    #plt.text(start,Input[start],"Start")
    #plt.plot([end,end],[Input[start],Input[end]],color='black')  
    #plt.text(end,Input[end],"end")
    
    plt.ylabel("Accumultion")
    plt.xlabel("days")
    plt.title("Onset and End Rain sesion")
    plt.legend()


year = 365
dic_year={}

fig =plt.figure(figsize=(15,10))
for i in range(1,31):
    aa=AA(B[(i-1)*year:i*year])
    #plotAA(aa,1980+i)
    plt.plot(aa, label=1980+i)  
    dic_year[1980+i]=np.argmin(aa)
plt.ylabel("Accumultion")
plt.xlabel("days")
plt.title("Onset and End Rain sesion")
plt.legend()
    


The AA compyted by Liebmann’s method
 
Dans cette parti nous avons étudiés la variabilité des OD de la prériode en question. 

In [ ]:
fig =plt.figure("variability years/days",figsize=(15,10))
plt.scatter(dic_year.keys(),dic_year.values())
plt.xlabel("Years")
plt.ylabel("Days")
plt.title("Interannual variability of onset dates")
plt.xticks(np.arange(1981,2011), rotation=60)


Y= np.array([i for i in dic_year.values()])
X=np.array([i for i in dic_year.keys()])
from scipy import stats
def predict(X,Y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, Y)
    return slope * np.array(X) + intercept
fitLine = predict(X,Y)
plt.plot(X, fitLine, c='r',label="Linear Regression")
plt.legend()

the previous image shows the onset date for each year between 1981 and 2020 for all area (Interannual variability of onset dates).

In term of average, we notice that the OD of the studied area is varying from year to year. In addition, we observe a progressive delay of OD.


#  Onsent for each grid point in the area

In [ ]:
def AA_Grid(Input):
    Moy=np.mean(Input,axis=0)
    aa=[]
    for i in range(1,366):
        aa.append(sum(Input[0:i]-Moy))
    return np.array(aa)

start_grid=[]

fig =plt.figure(figsize=(20,20))
for i in range(1,31):
    aa_grid=AA_Grid(A[(i-1)*year:i*year])
    start_grid.append(np.argmin(aa_grid,axis=0))
    plt.subplot(5,6,i)
    plt.imshow(start_grid[i-1],cmap=plt.cm.get_cmap('jet', len(np.unique(start_grid[i-1]))))
    plt.title(str(1980+i)+" Nb OD:"+str(len(np.unique(start_grid[i-1]))))
    plt.colorbar()


The previous figure shows the OD for each grid point in area for preriod 1981 until 2010.


We observe that the number of Onset dates (OD) on the studied area is very irregular from one year to another, ranging from 12 days (for the years 2005 and 2003) to 28 days (for the years 1989 and 2002).



In [ ]:
start_grid_avreg=np.mean(np.array(start_grid),axis=0,dtype=int)
fig =plt.figure(figsize=(15,15))
#plt.imshow(start_grid_avreg,cmap=plt.cm.get_cmap('jet', 20))#len(np.unique(start_grid_avreg))))
plt.imshow(start_grid_avreg,cmap=plt.cm.get_cmap('jet',len(np.unique(start_grid_avreg))))
plt.colorbar()
plt.title("Avrege Start Date")
plt.xticks(np.arange(21),("15E","","","","","15.5E","","","","","16E","","","","","16.5E","","","","","17E"))
plt.yticks(np.arange(41),("-9N","","","","","","","","","","-10N","","","","","","","","","","-11N","","","","","","","","","","-12N","","","","","","","","","","-13N"))


The previous figure shows the  avrege onset date for each grid point in area for preriod 1981 until 2010.

In term of average, we clearly observe the presence of a spatial gradient from north to south.

In [ ]:
import cdsapi
c = cdsapi.Client()
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'grib',
        'area': [
           -9.54,16.33,-9.56,16.35,
        ],
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
        ],
        'variable': 'total_precipitation',
    },
    'DataM.grib')


c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'grib',
        'area': [
           -12.76,15.72,-12.78,15.74,
        ],
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
        ],
        'variable': 'total_precipitation',
    },
    'DataH.grib')

In [ ]:
#fileM = '/home/az/Bureau/MFI_Test/DataM.grib' #put the correcte filename path 
#fileH = '/home/az/Bureau/MFI_Test/DataH.grib' #put the correcte filename path 

file=os.path.abspath(os.getcwd())
fileM=file+'/DataM.grib'
fileH=file+'/DataH.grib'

grM = pygrib.open(fileM)
grH = pygrib.open(fileH)
grM.rewind() # rewind the iterator
grH.rewind() # rewind the iterator
MListe_Value=[]
HListe_Value=[]

for msgM in grM:
    temps_valsM=msgM.values
    MListe_Value.append(temps_valsM)
    
for msgH in grH:
    temps_valsH=msgH.values
    HListe_Value.append(temps_valsH)

MAll_Value=np.array(MListe_Value)
HAll_Value=np.array(HListe_Value)

M_mm=id2precip2(MAll_Value)
H_mm=id2precip2(HAll_Value)   




In [ ]:
#%%
dic_yearM={}
fig =plt.figure('AA_Malanje')
for i in range(1,31):
    aa=AA(M_mm[(i-1)*year:i*year])
    plt.plot(aa,label=str(1980+i))
    dic_yearM[1980+i]=np.argmin(aa)
plt.legend()
plt.ylabel("Accumultion")
plt.xlabel("days")
plt.title("Anomalous Accumulation Malanje ")

#%%


In [ ]:
dic_yearH={}
fig =plt.figure('AA_Huambo')
for i in range(1,31):
    aa=AA(H_mm[(i-1)*year:i*year])
    plt.plot(aa,label=str(1980+i))
    dic_yearH[1980+i]=np.argmin(aa)
plt.legend()
plt.ylabel("Accumultion")
plt.xlabel("days")
plt.title("Anomalous Accumulation Huambo")    

#%%

In [ ]:
fig =plt.figure("variability years/days ",figsize=(15,10))
plt.scatter(dic_yearM.keys(),dic_yearM.values(),label="Malanje",color='r')
plt.scatter(dic_yearH.keys(),dic_yearH.values(),label="Huambo",,color='b')
plt.plot(dic_yearM.keys(),dic_yearM.values())
plt.plot(dic_yearH.keys(),dic_yearH.values())
plt.xlabel("Years")
plt.ylabel("Days")
plt.title("variability years/days")
plt.legend()
plt.xticks(np.arange(1981,2011), rotation=60)

The figure presente the inter-annual variability of onset dates for Manlanje (read) and Huambo (bleu).

We observe that the temporal variability of the onset date for the two cities Huambo and Malanje are highly correlated. Durring the period 1981-1988 and 2002-2010 the Malanje's Onset is detected before the Huambo's Onset,  on the other hand, for the period 1989- 2001 the Malanje's Onset is detected before the Huambo's.

